In [1]:
import pandas as pd
import numpy as np
import video.df as df
from video import vid
from video import df
from video import frame

import video.reader as r
import cv2 as cv
from video.reader import VideoReader


from typing import List, Union
from numpy.typing import ArrayLike

In [2]:
vid, fps = VideoReader.get_vid('videos/Banned Pokemon Seizure Scene.mp4', 
                conversion=cv.COLOR_BGR2HLS)

In [23]:
vid.shape

(785, 50, 50, 3)

In [5]:
width = vid.shape[2]
height = vid.shape[1]
frames = vid.shape[0]

In [28]:
width, height

(50, 50)

In [16]:
vid[0][0][0]

array([ 0, 36,  0], dtype=uint8)

In [8]:
vid.shape

(785, 50, 50, 3)

In [12]:
vid[0].reshape(-1, 3).shape

(2500, 3)

In [3]:
def get_red_mask(img):
    # mask for hue below 10 hue, lightness, saturation
    lb = np.array([0,50,50], dtype=np.uint8)
    ub = np.array([10,255,255], dtype=np.uint8)
    mask1 = cv.inRange(img, lb, ub)

    # # mask for hue above 340
    lb1 = np.array([170,50, 50], dtype=np.uint8)
    ub1 = np.array([180,255, 255], dtype=np.uint8)
    mask2 = cv.inRange(img, lb1, ub1)

    return mask1 | mask2

In [25]:
im1 = cv.imread('img/img40.jpg')

In [26]:
im1.shape

(400, 600, 3)

In [14]:
# create en empty array of integers
narr = np.empty((0,), dtype=np.uint8)
for v in vid:
    # get video frame
    frame = v.reshape(height, width, 3)
    # apply red mask
    mask = get_red_mask(frame)
    # append masked values as 1D array narr
    narr = np.concatenate([narr, mask.reshape(-1)])
# narr -> array whith masked red values. represent pixels of each video frame
# in the video. if the pixel is red it equals 1 else 0
narr = np.where(narr == 255, 1, 0)
# count the number of pixels in frame
total_pixels_per_frame = width * height
    

In [8]:
narr.shape[0] / total_pixels_per_frame

785.0

In [54]:
np.unique(narr)

array([0, 1])

In [15]:
# reshape the narr array to the shape of (frame, total pixels)
narr = narr.reshape(frames, height * width)

In [21]:
red_frames = 0
for n in narr:
    if n.sum() > total_pixels_per_frame / 4:
        red_frames += 1
print(red_frames)

145


In [25]:
def detect_red_frames_portion(video_path: str):
    '''
    detects what % of video frames have red color on more than 25% of the screen
    '''
    # get video and fps
    vid, fps = VideoReader.get_vid(video_path, 
                conversion=cv.COLOR_BGR2HLS)
    # get # of frames, width and height of each frame
    width = vid.shape[2]
    height = vid.shape[1]
    frames = vid.shape[0]

    # create en empty array of integers
    narr = np.empty((0,), dtype=np.uint8)
    for v in vid:
        # get video frame
        frame = v.reshape(height, width, 3)
        # apply red mask
        mask = get_red_mask(frame)
        # append masked values as 1D array narr
        narr = np.concatenate([narr, mask.reshape(-1)])
    # narr -> array whith masked red values. represent pixels of each video frame
    # in the video. if the pixel is red it equals 1 else 0
    narr = np.where(narr == 255, 1, 0)
    # count the number of pixels in frame
    total_pixels_per_frame = width * height
    
    # reshape the narr array to the shape of (frame, total pixels)
    narr = narr.reshape(frames, total_pixels_per_frame)

    red_frames = 0
    for n in narr:
        if n.sum() > total_pixels_per_frame / 4:
            red_frames += 1
    
    return round(red_frames / frames * 100, 2)

In [26]:
detect_red_frames_portion(video_path='videos/Banned Pokemon Seizure Scene.mp4')

18.47